##  DELTA LIVE TABLE-GOLD LAYER

In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *

## EXPECTATIONS FOR DATA QUALITY


In [0]:
expec_coaches = {
    "rule1": "code is not null",
    "rule2": "current is True"
            }


In [0]:
expec_nocs = {
    "rule1": "code is not null"
            }


In [0]:
expec_events = {
    "rule1": "event is not null"
        }



## COACHES-DLT PIPELINE


In [0]:
@dlt.table

def source_coaches():
    df = spark.readStream.table("olympics_catalog.silver_sch.coaches")
    return df

In [0]:
@dlt.view

def view_coaches():
    df = spark.readStream.table("LIVE.source_coaches")
    df.fillna("unknown")
    return df

In [0]:
@dlt.table
@dlt.expect_all(expec_coaches)
def coaches():
  df = spark.readStream.table("LIVE.view_coaches")
  return df

## NOCS DLT PIPELINE

In [0]:
@dlt.view

def view_nocs():
    
    df = spark.readStream.table("olympics_catalog.silver_sch.nocs")
    return df
    

In [0]:
@dlt.table
@dlt.expect_all_or_drop(expec_nocs)
def nocs():
    
    df = spark.readStream.table("LIVE.view_nocs")
    return df

## EVENTS DLT PIPELINE

In [0]:
@dlt.view

def view_events():
    df = spark.readStream.table("olympics_catalog.silver_sch.events")
    return df

In [0]:
@dlt.table

@dlt.expect_all(expec_events)
def events():
    df=spark.readStream.table("LIVE.view_events")

    return df

## CDC -APPLY CHANGES DLT

In [0]:
import dlt

In [0]:
@dlt.view

def source_athletes():
    df= spark.readStream.table("olympics_catalog.silver_sch.athletes")
    return df

In [0]:
dlt.create_streaming_table("athletes")

In [0]:
dlt.apply_changes(
    target = "athletes",
    source = "source_athletes",
    keys = ["Athlete_id"],
    sequence_by = col("height"),
    stored_as_scd_type = 1
)